In [31]:
import torchvision.models
import torch
import tensorrt as trt
from torch2trt import torch2trt

In [23]:
def cross_validate(model, model_trt, data, tolerance=0.01):
    y = model(data)
    y_trt = model_trt(data)
    max_error = torch.max(torch.abs(y - y_trt))
    if max_error > tolerance:
        raise RuntimeError('Model exceeds tolerance')

In [25]:
net = torch.nn.Sequential(*[
    torch.nn.Conv2d(3, 32, 3),
    torch.nn.ReLU(),
    torch.nn.BatchNorm2d(32)
]).eval().half().cuda()

data = torch.randn((1, 3, 224, 224)).cuda().half()

In [26]:
net_trt = torch2trt(net, [data], fp16_mode=True)

In [37]:
help(trt.INetworkDefinition.mark_output)

Help on instancemethod in module tensorrt.tensorrt:

mark_output(...)
    mark_output(self: tensorrt.tensorrt.INetworkDefinition, tensor: tensorrt.tensorrt.ITensor) -> None
    
    
    Mark a tensor as an output.
    
    :arg tensor: The tensor to mark.



In [27]:
cross_validate(net, net_trt, data)

RuntimeError: expected type torch.cuda.FloatTensor but got torch.cuda.HalfTensor